In [1]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    fr = open(fileName)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

import random
def selectJrand(i, m):
    j = i
    while(j==i):
        j = int (random.uniform(0, m))
    return j

def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
    return aj

In [2]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [3]:
dataArr, labelArr

([[3.542485, 1.977398],
  [3.018896, 2.556416],
  [7.55151, -1.58003],
  [2.114999, -0.004466],
  [8.127113, 1.274372],
  [7.108772, -0.986906],
  [8.610639, 2.046708],
  [2.326297, 0.265213],
  [3.634009, 1.730537],
  [0.341367, -0.894998],
  [3.125951, 0.293251],
  [2.123252, -0.783563],
  [0.887835, -2.797792],
  [7.139979, -2.329896],
  [1.696414, -1.212496],
  [8.117032, 0.623493],
  [8.497162, -0.266649],
  [4.658191, 3.507396],
  [8.197181, 1.545132],
  [1.208047, 0.2131],
  [1.928486, -0.32187],
  [2.175808, -0.014527],
  [7.886608, 0.461755],
  [3.223038, -0.552392],
  [3.628502, 2.190585],
  [7.40786, -0.121961],
  [7.286357, 0.251077],
  [2.301095, -0.533988],
  [-0.232542, -0.54769],
  [3.457096, -0.082216],
  [3.023938, -0.057392],
  [8.015003, 0.885325],
  [8.991748, 0.923154],
  [7.916831, -1.781735],
  [7.616862, -0.217958],
  [2.450939, 0.744967],
  [7.270337, -2.507834],
  [1.749721, -0.961902],
  [1.803111, -0.176349],
  [8.804461, 3.044301],
  [1.231257, -0.568573],

In [31]:
import numpy as np
from numpy import *

def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = np.mat(dataMatIn)  # 将数据集转换为NumPy矩阵
    labelMat = np.mat(classLabels).transpose()  # 将标签转换为列向量
    b = 0  # 初始化偏置项 b
    m, n = np.shape(dataMatrix)  # 获取数据集的样本数和特征数
    alphas = np.mat(np.zeros((m, 1)))  # 初始化拉格朗日乘子 alphas
    iter = 0  # 初始化迭代次数

    while (iter < maxIter):  # 外循环：迭代固定次数或直到没有alpha改变为止
        alphaPairsChanged = 0  # 记录每次迭代中alpha对的改变次数
        for i in range(m):  # 内循环：遍历每个样本点
            fXi = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[i, :].T)) + b  # 计算函数间隔
            Ei = fXi - float(labelMat[i])  # 计算函数间隔与实际类别之间的差值

            # 检查 alpha_i 是否满足 KKT 条件
            if ((labelMat[i] * Ei < -toler) and (alphas[i] < C)) or ((labelMat[i] * Ei > toler) and (alphas[i] > 0)):
                j = selectJrand(i, m)  # 随机选择 alpha_j 的索引
                fXj = float(np.multiply(alphas, labelMat).T * (dataMatrix * dataMatrix[j, :].T)) + b
                Ej = fXj - float(labelMat[j])

                # 保存更新前的 alpha 值，用于后面的 alpha 值修剪
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()

                # 计算 alpha_j 的取值范围
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])

                if L == H:
                    print("L==H")
                    continue

                eta = 2.0 * dataMatrix[i, :] * dataMatrix[j, :].T - dataMatrix[i, :] * dataMatrix[i, :].T - dataMatrix[j, :] * dataMatrix[j, :].T
                # 计算 alpha_j 的新值
                if eta >= 0:
                    print("eta>=0")
                    continue

                alphas[j] -= labelMat[j] * (Ei - Ej) / eta
                alphas[j] = clipAlpha(alphas[j], H, L)  # 对 alpha_j 进行修剪

                # 检查 alpha_j 是否有足够的变化
                if abs(alphas[j] - alphaJold) < 0.00001:
                    print("j 没有多大变化")
                    continue

                # 更新 alpha_i
                alphas[i] += labelMat[j] * labelMat[i] * (alphaJold - alphas[j])

                # 计算更新后的偏置项 b
                b1 = b - Ei - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[i, :].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[i, :] * dataMatrix[j, :].T
                b2 = b - Ej - labelMat[i] * (alphas[i] - alphaIold) * dataMatrix[i, :] * dataMatrix[j, :].T - labelMat[j] * (alphas[j] - alphaJold) * dataMatrix[j, :] * dataMatrix[j, :].T

                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2) / 2.0

                alphaPairsChanged += 1
                print("iter: %d i : %d, pairs changed %d" % (iter, i, alphaPairsChanged))

        if alphaPairsChanged == 0:
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)

    return b, alphas


In [6]:
b

matrix([[-3.83861634]])

In [7]:
alphas[alphas>0]

matrix([[0.12738006, 0.24141068, 0.36879074]])

In [4]:
from numpy import *

In [5]:
def kernelTrans(X, A, kTup): #calc the kernel or transform data to a higher dimensional space
    m,n = shape(X)
    K = mat(zeros((m,1)))
    if kTup[0]=='lin': K = X * A.T   #linear kernel
    elif kTup[0]=='rbf':
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = exp(K/(-1*kTup[1]**2)) #divide in NumPy is element-wise not matrix like Matlab
    else: raise NameError('Houston We Have a Problem -- \
    That Kernel is not recognized')
    return K

In [12]:
class optStruct:
    def __init__(self,dataMatIn, classLabels, C, toler, kTup):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        self.K = mat(zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
        
def calcEk(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b)
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJ(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEk(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]

In [7]:
def innerL(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print ("L==H"); return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j] #changed for kernel
        if eta >= 0: print ("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i] - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]- oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

In [10]:
def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print ("fullSet, iter: %d i:%d, pairs changed %d"% (iter,i,alphaPairsChanged)) 
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print ("non-bound, iter: %d i:%d, pairs changed %d"% (iter,i,alphaPairsChanged)) 
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print ("iteration number: %d" % iter)
    return oS.b,oS.alphas

In [14]:
dataArra, labelArr = loadDataSet('testSet.txt')

In [13]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
L==H
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
j not moving enough
fullSet, iter: 0 i:7, pairs changed 2
L==H
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
L==H
fullSet, iter: 0 i:10, pairs changed 2
L==H
fullSet, iter: 0 i:11, pairs changed 2
L==H
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
L==H
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
L==H
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
j not moving enough
fullSet, iter: 0 i:22, pairs changed 3
L==H
fullSet, iter: 0 i:23, 

In [14]:
b

matrix([[-2.89901748]])

In [15]:
alphas

matrix([[0.06961952],
        [0.        ],
        [0.        ],
        [0.0169055 ],
        [0.0169055 ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.0272699 ],
        [0.04522972],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.0272699 ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0

In [16]:
def calcWs(alphas,dataArr,classLabels):
    X = mat(dataArr); labelMat = mat(classLabels).transpose()
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*labelMat[i],X[i,:].T)
    return w

In [17]:
ws = calcWs(alphas, dataArr, labelArr)

In [18]:
ws

array([[ 0.65307162],
       [-0.17196128]])

In [19]:
datMat = mat(dataArr)
datMat[0] * mat(ws) + b

matrix([[-0.92555695]])

In [28]:
def testRbf(k1=1.3):
    dataArr,labelArr = loadDataSet('testSetRBF.txt')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, maxIter=10000, kTup= ('rbf', k1)) #C=200 important
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] #get matrix of only support vectors
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadDataSet('testSetRBF2.txt')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],('rbf', k1))
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m) )   

In [29]:
testRbf()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 6
fullSet, iter: 0 i:10, pairs changed 7
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 10
fullSet, iter: 0 i:16, pairs changed 11
fullSet, iter: 0 i:17, pairs changed 11
fullSet, iter: 0 i:18, pairs changed 12
fullSet, iter: 0 i:19, pairs changed 13
fullSet, iter: 0 i:20, pairs changed 13
fullSet, iter: 0 i:21, pairs changed 14
fullSet, iter: 0 i:22, pairs changed 14
fullSet, iter: 0 i:23, pairs changed 14
fullSet, iter: 0 i:24, pairs changed 15
j not moving enough
fullS

the training error rate is: 0.020000
the test error rate is: 0.090000


In [30]:
def img2vector(filename):
    returnVect = zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImages(dirName):
    from os import listdir
    hwLabels = []
    trainingFileList = listdir(dirName)           #load the training set
    m = len(trainingFileList)
    trainingMat = zeros((m,1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]     #take off .txt
        classNumStr = int(fileStr.split('_')[0])
        if classNumStr == 9: hwLabels.append(-1)
        else: hwLabels.append(1)
        trainingMat[i,:] = img2vector('%s/%s' % (dirName, fileNameStr))
    return trainingMat, hwLabels    

def testDigits(kTup=('rbf', 10)):
    dataArr,labelArr = loadImages('trainingDigits')
    b,alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, kTup)
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    svInd=nonzero(alphas.A>0)[0]
    sVs=datMat[svInd] 
    labelSV = labelMat[svInd];
    print ("there are %d Support Vectors" % shape(sVs)[0])
    m,n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1
    print ("the training error rate is: %f" % (float(errorCount)/m))
    dataArr,labelArr = loadImages('testDigits')
    errorCount = 0
    datMat=mat(dataArr); labelMat = mat(labelArr).transpose()
    m,n = shape(datMat)
    for i in range(m):
        kernelEval = kernelTrans(sVs,datMat[i,:],kTup)
        predict=kernelEval.T * multiply(labelSV,alphas[svInd]) + b
        if sign(predict)!=sign(labelArr[i]): errorCount += 1    
    print ("the test error rate is: %f" % (float(errorCount)/m) )

In [31]:
testDigits()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 2
fullSet, iter: 0 i:2, pairs changed 3
fullSet, iter: 0 i:3, pairs changed 4
fullSet, iter: 0 i:4, pairs changed 5
fullSet, iter: 0 i:5, pairs changed 6
fullSet, iter: 0 i:6, pairs changed 6
fullSet, iter: 0 i:7, pairs changed 7
fullSet, iter: 0 i:8, pairs changed 8
fullSet, iter: 0 i:9, pairs changed 9
fullSet, iter: 0 i:10, pairs changed 10
fullSet, iter: 0 i:11, pairs changed 10
fullSet, iter: 0 i:12, pairs changed 11
fullSet, iter: 0 i:13, pairs changed 12
fullSet, iter: 0 i:14, pairs changed 13
fullSet, iter: 0 i:15, pairs changed 14
fullSet, iter: 0 i:16, pairs changed 15
fullSet, iter: 0 i:17, pairs changed 16
j not moving enough
fullSet, iter: 0 i:18, pairs changed 16
fullSet, iter: 0 i:19, pairs changed 17
fullSet, iter: 0 i:20, pairs changed 18
fullSet, iter: 0 i:21, pairs changed 19
fullSet, iter: 0 i:22, pairs changed 20
fullSet, iter: 0 i:23, pairs changed 21
fullSet, iter: 0 i:24, pairs changed 22


L==H
fullSet, iter: 0 i:189, pairs changed 34
fullSet, iter: 0 i:190, pairs changed 34
fullSet, iter: 0 i:191, pairs changed 34
L==H
fullSet, iter: 0 i:192, pairs changed 34
fullSet, iter: 0 i:193, pairs changed 34
fullSet, iter: 0 i:194, pairs changed 34
fullSet, iter: 0 i:195, pairs changed 34
L==H
fullSet, iter: 0 i:196, pairs changed 34
L==H
fullSet, iter: 0 i:197, pairs changed 34
fullSet, iter: 0 i:198, pairs changed 35
fullSet, iter: 0 i:199, pairs changed 36
fullSet, iter: 0 i:200, pairs changed 37
fullSet, iter: 0 i:201, pairs changed 38
fullSet, iter: 0 i:202, pairs changed 39
fullSet, iter: 0 i:203, pairs changed 40
fullSet, iter: 0 i:204, pairs changed 41
fullSet, iter: 0 i:205, pairs changed 42
fullSet, iter: 0 i:206, pairs changed 43
fullSet, iter: 0 i:207, pairs changed 44
fullSet, iter: 0 i:208, pairs changed 45
fullSet, iter: 0 i:209, pairs changed 46
fullSet, iter: 0 i:210, pairs changed 47
fullSet, iter: 0 i:211, pairs changed 48
fullSet, iter: 0 i:212, pairs changed

fullSet, iter: 0 i:401, pairs changed 119
iteration number: 1
non-bound, iter: 1 i:0, pairs changed 1
non-bound, iter: 1 i:1, pairs changed 2
j not moving enough
non-bound, iter: 1 i:2, pairs changed 2
non-bound, iter: 1 i:3, pairs changed 3
j not moving enough
non-bound, iter: 1 i:4, pairs changed 3
non-bound, iter: 1 i:5, pairs changed 4
non-bound, iter: 1 i:7, pairs changed 5
j not moving enough
non-bound, iter: 1 i:8, pairs changed 5
j not moving enough
non-bound, iter: 1 i:10, pairs changed 5
j not moving enough
non-bound, iter: 1 i:13, pairs changed 5
j not moving enough
non-bound, iter: 1 i:14, pairs changed 5
j not moving enough
non-bound, iter: 1 i:15, pairs changed 5
j not moving enough
non-bound, iter: 1 i:16, pairs changed 5
j not moving enough
non-bound, iter: 1 i:17, pairs changed 5
j not moving enough
non-bound, iter: 1 i:19, pairs changed 5
j not moving enough
non-bound, iter: 1 i:20, pairs changed 5
j not moving enough
non-bound, iter: 1 i:21, pairs changed 5
j not mov

non-bound, iter: 2 i:22, pairs changed 0
j not moving enough
non-bound, iter: 2 i:23, pairs changed 0
j not moving enough
non-bound, iter: 2 i:28, pairs changed 0
j not moving enough
non-bound, iter: 2 i:32, pairs changed 0
j not moving enough
non-bound, iter: 2 i:39, pairs changed 0
j not moving enough
non-bound, iter: 2 i:42, pairs changed 0
j not moving enough
non-bound, iter: 2 i:44, pairs changed 0
j not moving enough
non-bound, iter: 2 i:47, pairs changed 0
j not moving enough
non-bound, iter: 2 i:49, pairs changed 0
j not moving enough
non-bound, iter: 2 i:53, pairs changed 0
j not moving enough
non-bound, iter: 2 i:54, pairs changed 0
j not moving enough
non-bound, iter: 2 i:57, pairs changed 0
j not moving enough
non-bound, iter: 2 i:59, pairs changed 0
j not moving enough
non-bound, iter: 2 i:63, pairs changed 0
j not moving enough
non-bound, iter: 2 i:64, pairs changed 0
j not moving enough
non-bound, iter: 2 i:77, pairs changed 0
j not moving enough
non-bound, iter: 2 i:81,

L==H
fullSet, iter: 3 i:27, pairs changed 0
j not moving enough
fullSet, iter: 3 i:28, pairs changed 0
j not moving enough
fullSet, iter: 3 i:29, pairs changed 0
j not moving enough
fullSet, iter: 3 i:30, pairs changed 0
fullSet, iter: 3 i:31, pairs changed 0
j not moving enough
fullSet, iter: 3 i:32, pairs changed 0
j not moving enough
fullSet, iter: 3 i:33, pairs changed 0
j not moving enough
fullSet, iter: 3 i:34, pairs changed 0
fullSet, iter: 3 i:35, pairs changed 0
fullSet, iter: 3 i:36, pairs changed 0
fullSet, iter: 3 i:37, pairs changed 0
fullSet, iter: 3 i:38, pairs changed 0
j not moving enough
fullSet, iter: 3 i:39, pairs changed 0
fullSet, iter: 3 i:40, pairs changed 0
j not moving enough
fullSet, iter: 3 i:41, pairs changed 0
j not moving enough
fullSet, iter: 3 i:42, pairs changed 0
fullSet, iter: 3 i:43, pairs changed 0
j not moving enough
fullSet, iter: 3 i:44, pairs changed 0
j not moving enough
fullSet, iter: 3 i:45, pairs changed 0
j not moving enough
fullSet, iter:

j not moving enough
fullSet, iter: 3 i:215, pairs changed 0
fullSet, iter: 3 i:216, pairs changed 0
fullSet, iter: 3 i:217, pairs changed 0
j not moving enough
fullSet, iter: 3 i:218, pairs changed 0
fullSet, iter: 3 i:219, pairs changed 0
fullSet, iter: 3 i:220, pairs changed 0
fullSet, iter: 3 i:221, pairs changed 0
fullSet, iter: 3 i:222, pairs changed 0
fullSet, iter: 3 i:223, pairs changed 0
fullSet, iter: 3 i:224, pairs changed 0
fullSet, iter: 3 i:225, pairs changed 0
fullSet, iter: 3 i:226, pairs changed 0
fullSet, iter: 3 i:227, pairs changed 0
fullSet, iter: 3 i:228, pairs changed 0
fullSet, iter: 3 i:229, pairs changed 0
fullSet, iter: 3 i:230, pairs changed 0
fullSet, iter: 3 i:231, pairs changed 0
j not moving enough
fullSet, iter: 3 i:232, pairs changed 0
fullSet, iter: 3 i:233, pairs changed 0
j not moving enough
fullSet, iter: 3 i:234, pairs changed 0
j not moving enough
fullSet, iter: 3 i:235, pairs changed 0
j not moving enough
fullSet, iter: 3 i:236, pairs changed 0


L==H
fullSet, iter: 3 i:398, pairs changed 0
fullSet, iter: 3 i:399, pairs changed 0
fullSet, iter: 3 i:400, pairs changed 0
j not moving enough
fullSet, iter: 3 i:401, pairs changed 0
iteration number: 4
there are 131 Support Vectors
the training error rate is: 0.000000
the test error rate is: 0.010753


In [51]:

'''#######********************************
Non-Kernel VErsions below
'''#######********************************

class optStructK:
    def __init__(self,dataMatIn, classLabels, C, toler):  # Initialize the structure with the parameters 
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m,1)))
        self.b = 0
        self.eCache = mat(zeros((self.m,2))) #first column is valid flag
        
def calcEkK(oS, k):
    fXk = float(multiply(oS.alphas,oS.labelMat).T*(oS.X*oS.X[k,:].T)) + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
        
def selectJK(i, oS, Ei):         #this is the second choice -heurstic, and calcs Ej
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.m)
        Ej = calcEk(oS, j)
    return j, Ej

def updateEkK(oS, k):#after any alpha has changed update the new value in the cache
    Ek = calcEk(oS, k)
    oS.eCache[k] = [1,Ek]
        
def innerLK(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.tol) and (oS.alphas[i] > 0)):
        j,Ej = selectJ(i, oS, Ei) #this has been changed from selectJrand
        alphaIold = oS.alphas[i].copy(); alphaJold = oS.alphas[j].copy();
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H: print ("L==H"); return 0
        eta = 2.0 * oS.X[i,:]*oS.X[j,:].T - oS.X[i,:]*oS.X[i,:].T - oS.X[j,:]*oS.X[j,:].T
        if eta >= 0: print ("eta>=0"); return 0
        oS.alphas[j] -= oS.labelMat[j]*(Ei - Ej)/eta
        oS.alphas[j] = clipAlpha(oS.alphas[j],H,L)
        updateEk(oS, j) #added this for the Ecache
        if (abs(oS.alphas[j] - alphaJold) < 0.00001): print ("j not moving enough"); return 0
        oS.alphas[i] += oS.labelMat[j]*oS.labelMat[i]*(alphaJold - oS.alphas[j])#update i by the same amount as j
        updateEk(oS, i) #added this for the Ecache                    #the update is in the oppostie direction
        b1 = oS.b - Ei- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[i,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[i,:]*oS.X[j,:].T
        b2 = oS.b - Ej- oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.X[i,:]*oS.X[j,:].T - oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.X[j,:]*oS.X[j,:].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): oS.b = b2
        else: oS.b = (b1 + b2)/2.0
        return 1
    else: return 0

def smoPK(dataMatIn, classLabels, C, toler, maxIter):    #full Platt SMO
    oS = optStruct(mat(dataMatIn),mat(classLabels).transpose(),C,toler)
    iter = 0
    entireSet = True; alphaPairsChanged = 0
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:   #go over all
            for i in range(oS.m):        
                alphaPairsChanged += innerL(i,oS)
                print ("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        else:#go over non-bound (railed) alphas
            nonBoundIs = nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i,oS)
                print ("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
            iter += 1
        if entireSet: entireSet = False #toggle entire set loop
        elif (alphaPairsChanged == 0): entireSet = True  
        print ("iteration number: %d" % iter)
    return oS.b,oS.alphas